In [4]:
import pandas as pd
import numpy as np

budgets_raw = pd.read_csv("Budgets_raw.csv", sep="|")

In [6]:
imdb_title_basics = pd.read_csv("title.basics.tsv", sep="\t", low_memory=False).query("titleType == 'movie'")

In [31]:
# separate year from release date and add year to own column
budgets_raw["year"] = budgets_raw["Release date"].apply(lambda x: x[-4:])

In [ ]:
#Load primary titles and original titles to own dataframes
imdb_primary = imdb_title_basics[["tconst", "primaryTitle", "startYear"]].rename(columns={"primaryTitle":"Title", "startYear":"year"}).dropna(subset="Title")
imdb_original = imdb_title_basics[["tconst", "originalTitle", "startYear"]].rename(columns={"originalTitle":"Title", "startYear":"year"}).dropna(subset="Title")

In [105]:
# Filter duplicates from budgets into own separate dataframe
budgets = budgets_raw.loc[~budgets_raw.duplicated(subset="Title", keep=False)].copy()
budgets_duplicates = budgets_raw.loc[budgets_raw.duplicated(subset="Title", keep=False)].copy()

In [106]:
# Join budgets with primary titles on title name and year
budgets1 = pd.merge(budgets, imdb_primary, on=["Title", "year"], how="left")

In [107]:
# Remove duplicated titles
removed1 = budgets1.loc[budgets1.duplicated(subset="Title", keep=False)]
budgets1 = budgets1.loc[~budgets1.duplicated(subset="Title", keep=False)]


In [ ]:
removed1.to_csv("removed duplicated titles.csv", index=False)

In [161]:
# list of index numbers for missing titles
missing_titles_indexes = budgets1.loc[(budgets1["tconst"].isna()) & (budgets1["year"]!="nown")].index


In [158]:

# from difflib import SequenceMatcher

# for index in missing_titles_indexes:
#     # threshold for ratio
#     best = 0.9
#     match_found = False

#     title_name = budgets1.loc[index, "Title"]
#     title_year = budgets1.loc[index, "year"]

#     imdb_titles = imdb_original.loc[(imdb_original["year"] == str(int(title_year)-1)) | (imdb_original["year"] == str(int(title_year)+1)) | (imdb_original["year"] == title_year), "Title"]
#     imdb_indexes = imdb_titles.index

#     for imdb_i, title in zip(imdb_indexes, imdb_titles):
#         ratio = SequenceMatcher(a=title_name, b=title).ratio()
#         if ratio > best:
#             match_found = True
#             best = ratio
#             best_imdb_index = imdb_i

#     if match_found:
#         #budgets1.loc[index, "Title"] = imdb_primary.loc[best_imdb_index, "Title"]
#         budgets1.loc[index, "tconst"] = imdb_primary.loc[best_imdb_index, "tconst"]



In [159]:
budgets1.to_csv("budgets.csv")

In [164]:
budgets1["tconst"].to_csv("imdb_codes.csv", index=False)

In [163]:
budgets1.loc[missing_titles_indexes].to_csv("budgets_missing_tconst.csv")